Collegamento con il database pymongo

In [120]:
import pymongo
from pymongo import MongoClient
client=MongoClient('mongodb://localhost:27017')
db=client.prova
UsersCollection=db.airport
db.airport.drop()

Importazione del file csv 

In [121]:
import csv
csvfile = open('C:/Users/glori/OneDrive/Desktop/dataman/RoutesIntegrato.csv', 'r')
reader = csv.DictReader( csvfile )

Definizione dello colonne

In [122]:
diz={'Airline':0,'Airline_Code':0,'Departure':0,'Arrivals':0,'Aircraft':0,'Distance':0}

Inserimento dei dati all'interno dello schema

In [123]:
for each in reader:
    row={}
    for field in diz:
            if field=='Departure':
                row[field]={'Departure':0,'Country_x':0,'Country_Code_x':0,'City_x':0,'Airport_Name_x':0}
                for key in row[field]:
                    row[field][key]=each[key]
            if field=='Arrivals':
                row[field]={'Arrivals':0,'Country_y':0,'Country_Code_y':0,'City_y':0,'Airport_Name_y':0}
                for key in row[field]:
                    row[field][key]=each[key]
            if field=='Airline':
                row[field]=each[field]
            if field=='Airline_Code':
                row[field]=each[field]
            if field=='Aircraft':
                row[field]=each[field]
            if field=='Distance':
                row[field]=each[field]
   
    db.airport.insert_one(row)


Compagnie aeree che hanno volato di più

In [190]:
import pandas as pd

agg_result= db.airport.aggregate(
    [{
    "$group" : 
        {"_id" : "$Airline",  
         "total" : {"$sum" : 1}
         }}
    ])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Airline', 'Number of Fly'])
df=df.sort_values('Number of Fly',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])


           Airline  Number of Fly
0          Ryanair           3190
1         Wizz Air           1145
2          easyJet            992
3          Vueling            428
4        Lufthansa            362
5        Eurowings            263
6             Jet2            236
7  British Airways            228
8      TUI Airways            214
9       Air France            187


Chilometri percorsi da ogni compagnia

In [191]:
agg_result=db.airport.aggregate([{
  "$group": {
    "_id": "$Airline",
    "sum_distance": {
      "$sum":  {'$convert': { 'input': '$Distance', 'to': 'int' }}
    }
  }
},{
  "$project": {
    "Airline": "$_id",
    "sum_distance": 1,
    "_id": 0
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
d = pd.DataFrame(list, columns = ['km', 'Airline'])
d=d.sort_values('km',ascending=False)
d.index = d.reset_index(drop=True).index 
print(d.iloc[0:10,:])


        km          Airline
0  4493124          Ryanair
1  1715527         Wizz Air
2  1330670          easyJet
3   550612          Vueling
4   498151             Jet2
5   417474      TUI Airways
6   381068        Eurowings
7   379716        Lufthansa
8   245697  British Airways
9   220691        Transavia


Normalizzazione dei chilometri percorsi in base al numero di voli per ogni compagnia aerea

In [193]:
for i in range(len(df)):
    for j in range(len(d)):
        if (df.loc[i,'Airline']==d.loc[j,'Airline']):
                df.loc[i,'km_norm']=d.loc[j,'km']/df.loc[i,'Number of Fly']
                
                
df=df.sort_values('km_norm',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

               Airline  Number of Fly      km_norm
0     Georgian Airways              6  2972.666667
1               Condor             72  2635.069444
2           Smartwings              5  2520.400000
3           Air Astana              2  2433.500000
4           SunExpress             53  2218.339623
5                 Jet2            236  2110.809322
6           Icelandair             49  2074.000000
7  Azerbaijan Airlines             23  2060.826087
8          TUI Airways            214  1950.813084
9        Edelweiss Air             31  1902.258065


Aircraft più utilizzati

In [126]:
agg_result= db.airport.aggregate(
    [{
    "$group" : 
        {"_id" : "$Aircraft",  
         "total" : {"$sum" : 1}
         }}
    ])

list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Aircraft', 'voli'])
df=df.sort_values('voli',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:4,:])


  Aircraft  voli
0      73H  3512
1      320  2044
2            766
3      319   687


Numeri di partenze per ogni nazione

In [195]:
agg_result= db.airport.aggregate(
    [{
    "$group" : 
        {"_id" : "$Departure.Country_x",  
         "total" : {"$sum" : 1}
         }}
    ])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Country', 'Number of Fly'])
df=df.sort_values('Number of Fly',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

          Country  Number of Fly
0           Spain           1713
1  United Kingdom           1426
2           Italy           1181
3         Germany            833
4          France            755
5          Poland            378
6        Portugal            374
7          Greece            307
8     Netherlands            294
9     Switzerland            266


Numero di voli da ogni aereoporto

In [129]:
agg_result=db.airport.aggregate([
        {"$group" : {"_id":"$Departure.Departure", "count":{"$sum":1}}}
    ])  


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Airport', 'Departure'])
df=df.sort_values('Departure',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

  Airport  Departure
0     MAD        199
1     AMS        198
2     BCN        191
3     DUB        172
4     VIE        165
5     MAN        163
6     LGW        148
7     LPA        146
8     TFS        144
9     AGP        144


Numero di aereoporti attivi per ogni stato

In [130]:
agg_result=db.airport.aggregate([{
  "$group": {
    "_id": {
      "Airport_Code": "$Departure.Departure",
      "Country": "$Departure.Country_x"
    },
    "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Airport_Code": "$_id.Airport_Code",
    "Country": "$_id.Country",
    "count": 1,
    "_id": 0
  }
},{
  "$group": {
    "_id": "$Country",
    "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Country": "$_id",
    "count": 1,
    "_id": 0
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Airport', 'Country'])
df=df.sort_values('Airport',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Airport             Country
0       43              France
1       42      United Kingdom
2       37              Greece
3       37               Spain
4       34               Italy
5       20             Germany
6       18              Sweden
7       17             Finland
8       13  Russian Federation
9       13              Poland


Quali sono le mete favorite delle nazioni con più voli

Italy

In [133]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "Italy"
    
  }
},{
  "$group": {
    "_id": "$Arrivals.Country_y",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Arrivals": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Arrivals', 'Airport'])
df=df.sort_values('Arrivals',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Arrivals         Airport
0       346           Italy
1       132           Spain
2       117  United Kingdom
3        80         Germany
4        71          France
5        62         Romania
6        55          Poland
7        33     Netherlands
8        31         Albania
9        27         Belgium


Spain

In [140]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "Spain"
    
  }
},{
  "$group": {
    "_id": "$Arrivals.Country_y",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Arrivals": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Arrivals', 'Airport'])
df=df.sort_values('Arrivals',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Arrivals         Airport
0       570           Spain
1       305  United Kingdom
2       163         Germany
3       132           Italy
4       117          France
5        62     Netherlands
6        45        Portugal
7        45     Switzerland
8        41         Ireland
9        37         Belgium


United Kingdom

In [196]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "United Kingdom"
    
  }
},{
  "$group": {
    "_id": "$Arrivals.Country_y",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Arrivals": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Arrivals', 'Airport'])
df=df.sort_values('Arrivals',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Arrivals         Airport
0       306           Spain
1       260  United Kingdom
2       117           Italy
3       108          France
4        88          Poland
5        63        Portugal
6        62         Germany
7        59         Ireland
8        56         Austria
9        48     Switzerland


Compagnie aeree più usate in Italia

In [143]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "Italy"
    
  }
},{
  "$group": {
    "_id": "$Airline",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Airline": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Fly', 'Airline'])
df=df.sort_values('Fly',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Fly       Airline
0  599       Ryanair
1  166      Wizz Air
2  102       easyJet
3   78   ITA Airways
4   31       Vueling
5   23       Volotea
6   15     Lufthansa
7   14  Air Dolomiti
8   13   TUI Airways
9   11     Eurowings


Spagna

In [136]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "Spain"
    
  }
},{
  "$group": {
    "_id": "$Airline",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Airline": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Fly', 'Airline'])
df=df.sort_values('Fly',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Fly          Airline
0  568          Ryanair
1  266          Vueling
2  145           Iberia
3  122          easyJet
4   73  Binter Canarias
5   64         Wizz Air
6   56       Air Europa
7   55             Jet2
8   51          Volotea
9   46      TUI Airways


United Kingdom

In [137]:
agg_result=db.airport.aggregate([{
  "$match": {
    "Departure.Country_x": "United Kingdom"
    
  }
},{
  "$group": {
    "_id": "$Airline",
      "count": {
      "$sum": 1
    }
  }
},{
  "$project": {
    "Airline": "$_id",
    "_id": 0,
      "count":1
  }
}])


list=[]
for i in agg_result:
    l=[]
    for key in i:
        l.append(i[key])
    list.append(l)
df = pd.DataFrame(list, columns = ['Fly', 'Airline'])
df=df.sort_values('Fly',ascending=False)
df.index = df.reset_index(drop=True).index 
print(df.iloc[0:10,:])

   Fly          Airline
0  425          Ryanair
1  302          easyJet
2  125  British Airways
3  112             Jet2
4  100      TUI Airways
5   97         Loganair
6   59         Wizz Air
7   35       Aer Lingus
8   29            Flybe
9   22          Vueling
